![](https://seeklogo.com/images/N/nba-75-years-logo-7E38B318A3-seeklogo.com.png)

# **Initial EDA**
## CPSC322 Final Project
### Everett Johnson
### Luke Martin
### April 18, 2022
### NBA Team Success Predictor

In [1]:
import matplotlib.pyplot as plt
import os
from mysklearn.mypytable import MyPyTable
import mysklearn.myevaluation as myeval
from mysklearn.myclassifiers import MyKNeighborsClassifier, MyNaiveBayesClassifier, MyDummyClassifier, MyDecisionTreeClassifier
from mysklearn.myensembleclassifiers import MyRandomForestClassifier

file_loc = os.path.join("input_data", "processed_data", "team_info.csv")
team_data = MyPyTable().load_from_file(file_loc)

win_percentage = team_data.get_column("Win Percentage")
#plt.figure()
#plt.hist(win_percentage, bins=5)
#plt.show()

file_loc = os.path.join("input_data", "processed_data", "team_stats.csv")
team_stats = MyPyTable().load_from_file(file_loc)

jppg = team_stats.get_column("JPPG")
#plt.figure()
#plt.hist(jppg, bins=10)
#plt.show()
#
trb = team_stats.get_column("TRB")
#plt.figure()
#plt.title("TRB")
#plt.hist(trb, bins=5)
#plt.show()
#
ast = team_stats.get_column("AST")
#plt.figure()
#plt.title("AST")
#plt.hist(ast, bins=5)
#plt.show()


In [2]:
y = team_stats.get_column("Success")
team_stats.drop_column("Success")
team_stats.drop_column("Team")
team_stats.drop_column("Season")
print(team_stats.column_names)
X = team_stats.data

['JPPG', 'TRB', 'AST', 'FG', '3FG', 'STL', 'BLK']


In [3]:
from tabulate import tabulate
knn = MyKNeighborsClassifier(10)
print("KFold Acc:", myeval.kfold_accuracy(X, y, knn, n_splits=10, random_state=0))
print(tabulate(myeval.kfold_confusion_matrix(X, y, [1, 2, 3, 4, 5], knn, n_splits=10, random_state=0)))
dummy = MyDummyClassifier()
print("Dummy Acc:", myeval.kfold_accuracy(X, y, dummy, n_splits=10,random_state=0))

KFold Acc: 0.4116997792494481
-  --  --  ---  ---  --  -----  ---------------
    1   2    3    4   5  Total  Recognition (%)
1  11  23   22    1   0  57     19
2  10  60   91   24   2  187    32
3   3  61  157   77   8  306    51
4   0  13  113  137  11  274    50
5   1   2   27   44   8  82     10
-  --  --  ---  ---  --  -----  ---------------
Dummy Acc: 0.33774834437086093


In [4]:
from tabulate import tabulate
forest = MyRandomForestClassifier(100, 10, 3)
print("Forest Acc:", myeval.kfold_accuracy(X, y, forest, n_splits=3, random_state=1))
print(tabulate(myeval.kfold_confusion_matrix(X, y, [1, 2, 3, 4, 5], forest, n_splits=3, random_state=0)))

Forest Acc: 0.3973509933774834
-  -  --  ---  ---  -  -----  ---------------
   1   2    3    4  5  Total  Recognition (%)
1  1  27   25    4  0  57     2
2  1  50   97   39  0  187    27
3  1  51  165   89  0  306    54
4  0  14  110  148  2  274    54
5  0   2   25   53  2  82     2
-  -  --  ---  ---  -  -----  ---------------


In [5]:
from tabulate import tabulate
tree = MyDecisionTreeClassifier()

header = ["att" + str(i) for i in range(len(X[0]))] + ["class"]
# storing data in MyPyTable object for access to functions there
train_data = [X[i] + [y[i]] for i in range(len(X))]
team_data_ii = MyPyTable(header, train_data)
attrib_domains = {}
for attribute in header:
    attrib_domains[attribute] = team_data_ii.get_atttribute_domain(attribute)
tree.attribute_domains = attrib_domains

print("Tree Acc:", myeval.kfold_accuracy(X, y, tree, n_splits=3, random_state=0))
print(tabulate(myeval.kfold_confusion_matrix(X, y, [1, 2, 3, 4, 5], tree, n_splits=3)))

Tree Acc: 0.38962472406181015
-  -  --  ---  --  -  -----  ---------------
   1   2    3   4  5  Total  Recognition (%)
1  0  33   22   2  0  57     0
2  0  67  108  12  0  187    36
3  4  52  207  42  1  306    68
4  1  11  165  90  7  274    33
5  0   1   36  42  3  82     4
-  -  --  ---  --  -  -----  ---------------


In [6]:
from tabulate import tabulate
nb = MyNaiveBayesClassifier()
print("NB Acc:", myeval.kfold_accuracy(X, y, nb, n_splits=3, random_state=0))
print(tabulate(myeval.kfold_confusion_matrix(X, y, [1, 2, 3, 4, 5], nb, n_splits=3, random_state=0)))

NB Acc: 0.42935982339955847
-  --  --  ---  ---  --  -----  ---------------
    1   2    3    4   5  Total  Recognition (%)
1  23  16   13    5   0  57     40
2  25  63   75   22   2  187    34
3  16  55  149   80   6  306    49
4   1  13   94  137  29  274    50
5   0   3   13   49  17  82     21
-  --  --  ---  ---  --  -----  ---------------
